# Comparing Volume and Liquidity Between 3Pool and FraxBP Curve Metapools

This analysis dynamically locates all metapools for the 3pool and fraxbp on curve, then performs an analysis of their TVL (segmented by whether or not liquidity is in the base pool lp token or the paired asset) and monthly volume.

In [147]:
import json 
import os
import logging 
import asyncio
import logging

from typing import List
from datetime import timedelta
from pathlib import Path 
from pprint import PrettyPrinter

from subgrounds import Subgrounds
from web3 import Web3
from pycoingecko import CoinGeckoAPI

from prefect import task, flow 
from prefect.client import get_client
from prefect.tasks import task_input_hash

import pandas as pd 
import numpy as np 
import altair as alt 
import missingno as miss

from subgrounds.pagination import ShallowStrategy

from flywheel_util.constants import (
    colors_24,
    colors_28, 
    addresses, 
    url_infura, 
    url_snapshot, 
    url_subgraphs, 
    snapshot_api_max_records_per_request, 
    snapshot_api_max_skip,
)
from flywheel_util.tasks.general import df_to_sql
from flywheel_util.utils.util import (
    ddf, 
    first_row, 
    query_attrs,
    recursive_index_merge, 
    remove_prefix,
    df_cols_change_prefix, 
    df_cols_camel_to_snake, 
)

# logging.basicConfig(level=logging.DEBUG)

pp = PrettyPrinter().pprint

# alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [148]:
# w3 = Web3(Web3.HTTPProvider(url_infura))

In [149]:
sg = Subgrounds()
sg_curve_vol = sg.load_subgraph(url_subgraphs.convex.curve_vol_mainnet)

In [150]:
url_subgraphs.convex.curve_vol_mainnet

'https://api.thegraph.com/subgraphs/name/convex-community/volume-mainnet'

In [151]:
@task(persist_result=True, cache_key_fn=task_input_hash, cache_expiration=timedelta(days=1), tags=["curve_subgraph"])
def query_pools(metapool_lp_token: str): 
    # Get all metapools for a particular pool 
    query_pools = sg_curve_vol.Query.pools(
        first=1000, where={"coins_contains": [metapool_lp_token]}
    )
    dfs = sg.query_df([
        query_pools.id, 
        query_pools.symbol, 
        query_pools.coins, 
        query_pools.coinNames, 
    ])
    return dfs 
    
@task 
def process_pools(dfs): 
    df_pool_coin = recursive_index_merge(dfs) 
    df_pool_coin = df_cols_change_prefix(df_pool_coin, "pools_", "pool_")
    df_pool_coin = df_pool_coin.rename(columns={"pool_coins": "coin", "pool_coinNames": "coin_name"})
    return df_pool_coin

@task(persist_result=True, cache_key_fn=task_input_hash, cache_expiration=timedelta(days=1), tags=["curve_subgraph"])
def query_pool_snaps(pool_ids: List[str]): 
    query_snaps = sg_curve_vol.Query.dailyPoolSnapshots(
        first=30000, orderBy="timestamp", orderDirection="desc", where={"pool_in": pool_ids}
    )
    dfs = sg.query_df([
        query_snaps.pool.coins, 
        query_snaps.pool.id, 
        query_snaps.timestamp, 
        query_snaps.reservesUSD, 
    ], pagination_strategy=ShallowStrategy)
    return dfs 

@task 
def process_pool_snaps(dfs): 
    df_pool_snaps = recursive_index_merge(dfs) 
    df_pool_snaps = remove_prefix(df_pool_snaps, "dailyPoolSnapshots_")
    df_pool_snaps = df_cols_camel_to_snake(df_pool_snaps)
    df_pool_snaps = df_pool_snaps.rename(columns={"pool_coins": "coin"})
    df_pool_snaps.timestamp = pd.to_datetime(df_pool_snaps.timestamp, unit='s')
    return df_pool_snaps

@task(persist_result=True, cache_key_fn=task_input_hash, cache_expiration=timedelta(days=1), tags=["curve_subgraph"])
def query_pool_vol(pool_ids: List[str]): 
    query_vol = sg_curve_vol.Query.swapVolumeSnapshots(
        first=30000, orderBy="timestamp", orderDirection="desc", where={"period": 86400, "pool_in": pool_ids}
    )
    df_pool_vol = sg.query_df([
        query_vol.pool.id,
        query_vol.timestamp, 
        query_vol.volumeUSD, 
    ], pagination_strategy=ShallowStrategy)
    return df_pool_vol

@task 
def process_pool_vol(df_pool_vol): 
    df_pool_vol = remove_prefix(df_pool_vol, "swapVolumeSnapshots_")
    df_pool_vol = df_cols_camel_to_snake(df_pool_vol)
    df_pool_vol.timestamp = pd.to_datetime(df_pool_vol.timestamp, unit='s')
    return df_pool_vol
 
@flow
def flow_metapool(metapool_lp_token: str):

    dfs_pool_coin = query_pools(metapool_lp_token)  
    df_pool_coin = process_pools(dfs_pool_coin)
    
    pool_ids = df_pool_coin.pool_id.unique().tolist()
    pool_batch_size = 4
    i = 0 
    futures_snaps = []
    futures_vol = []
    while i < len(pool_ids):
        pool_id_batch = pool_ids[i:min(i+pool_batch_size, len(pool_ids))]
        i += pool_batch_size
        futures_snaps.append(query_pool_snaps.submit(pool_id_batch))
        futures_vol.append(query_pool_vol.submit(pool_id_batch))
    
    dfs_snaps = [f.result() for f in futures_snaps]
    dfs_vols = list(filter(lambda df: not df.empty, [f.result() for f in futures_vol]))
    
    dfs_snaps = list(map(lambda d: process_pool_snaps(d), dfs_snaps))
    dfs_vols = list(map(lambda d: process_pool_vol(d), dfs_vols))
            
    df_pool_snaps = pd.concat(dfs_snaps)
    df_pool_vol = pd.concat(dfs_vols)
    
    df_reserves = df_pool_coin.merge(df_pool_snaps, on=['pool_id', 'coin'])
    df_volume = df_pool_coin[['pool_id', 'pool_symbol']].drop_duplicates().merge(df_pool_vol, on=['pool_id'])

    return {"reserves": df_reserves, "volume": df_volume} 
    
@flow 
def flow_fraxbp_vs_3pool(): 
    results = {}
    for key in [
        '3crv', 
        'crvfrax'
    ]: 
        metapool_lp_token = addresses.token[key]
        results[key] = flow_metapool(metapool_lp_token)
    return results


/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/prefect/tasks.py:206: UserWarning: A task named 'query_pools' and defined at '/var/folders/5c/hf9_blgj3jbbxxvkw_s9snsw0000gn/T/ipykernel_94847/2663358337.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/prefect/tasks.py:206: UserWarning: A task named 'process_pools' and defined at '/var/folders/5c/hf9_blgj3jbbxxvkw_s9snsw0000gn/T/ipykernel_94847/2663358337.py:15' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/prefect/tasks.py:206: UserWarning: A task named 'query_pool_snaps' and defined at '/var/folders/5c/hf9_blgj3jbbxxvkw_s9snsw0000gn/T/ipykernel_94847/2663358337.py:22' conflicts with another tas

In [152]:
async with get_client() as client:
    # set a concurrency limit of 10 on the 'small_instance' tag
    limit_id = await client.create_concurrency_limit(tag="curve_subgraph", concurrency_limit=16)

data = flow_fraxbp_vs_3pool()

10:29:00.453 | INFO    | prefect.engine - Created flow run 'snobbish-collie' for flow 'flow-fraxbp-vs-3pool'

/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/prefect/tasks.py:206: UserWarning: A task named 'flow-metapool' and defined at '/var/folders/5c/hf9_blgj3jbbxxvkw_s9snsw0000gn/T/ipykernel_94847/2663358337.py:63' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


10:29:00.532 | INFO    | Flow run 'snobbish-collie' - Created subflow run 'magnificent-leopard' for flow 'flow-metapool'

10:29:00.557 | INFO    | Flow run 'magnificent-leopard' - Created task run 'query_pools-66de95c5-0' for task 'query_pools'

10:29:00.558 | INFO    | Flow run 'magnificent-leopard' - Executing 'query_pools-66de95c5-0' immediately...

...

10:29:48.130 | INFO    | Flow run 'nickel-iguana' - Executing 'process_pool_vol-f3874c79-12' immediately...

10:29:48.151 | INFO    | Task run 'process_pool_vol-f3874c79-12' - Finished in state Completed()

10:29:48.169 | INFO    | Flow run 'nickel-iguana' - Finished in state Completed()

10:29:48.177 | INFO    | Flow run 'snobbish-collie' - Finished in state Completed()

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x2aa941900>


In [153]:
data_3pool = data['3crv']
df_3pool_reserves = data_3pool['reserves']
df_3pool_volume = data_3pool['volume']

data_fraxbp = data['crvfrax'] 
df_fraxbp_reserves = data_fraxbp['reserves']
df_fraxbp_volume = data_fraxbp['volume']

In [174]:
def chart_metapool_tvl(df, coin_name):
    df_mp = df.loc[df.coin_name == coin_name].groupby("timestamp")['reserves_usd'].sum().reset_index()
    df_mp['segment'] = coin_name
    df_mp_paired = df.loc[df.coin_name != coin_name].groupby("timestamp")['reserves_usd'].sum().reset_index()
    df_mp_paired['segment'] = 'paired asset'
    df_full = pd.concat([df_mp, df_mp_paired])
    return (
        alt.Chart(df_full)
        .mark_area()
        .encode(
            x="timestamp:T", 
            y="reserves_usd:Q", 
            color="segment:N", 
            tooltip=["timestamp", alt.Tooltip("reserves_usd", format="$,d"), "segment"]
        )
    )

In [175]:
df_fraxbp_last = df_fraxbp_reserves.copy()
df_fraxbp_last['max_timestamp'] = df_fraxbp_last.groupby(['pool_symbol'])['timestamp'].transform("max")
df_fraxbp_last = df_fraxbp_last.loc[df_fraxbp_last.timestamp == df_fraxbp_last.max_timestamp]

df_3pool_last = df_3pool_reserves.copy()
df_3pool_last['max_timestamp'] = df_3pool_last.groupby(['pool_symbol'])['timestamp'].transform("max")
df_3pool_last = df_3pool_last.loc[df_3pool_last.timestamp == df_3pool_last.max_timestamp]

tvl_paired_fraxbp = df_fraxbp_last.loc[df_fraxbp_last.coin_name != 'crvFRAX'].reserves_usd.sum()
tvl_paired_3pool = df_3pool_last.loc[df_3pool_last.coin_name != '3Crv'].reserves_usd.sum()
print(f"TVL paired against crvFRAX across FraxBP metapools: ${tvl_paired_fraxbp:,}")
print(f"TVL paired against 3Crv across 3Pool metapools: ${tvl_paired_3pool:,}")
print(f"Ratio of paired liquidity of FraxBP to 3Pool: {100 * (tvl_paired_fraxbp / tvl_paired_3pool):.1f}%")

alt.hconcat(
    (
        chart_metapool_tvl(
            df_fraxbp_reserves, 
            'crvFRAX'
        )
        .properties(title="Reserves: FraxBP Metapools")
    ), 
    (
        chart_metapool_tvl(
            df_3pool_reserves.loc[df_3pool_reserves.timestamp >= df_fraxbp_reserves.timestamp.min()], 
            '3Crv'
        )
        .properties(title="Reserves: 3Pool Metapools")
    ),
)

TVL paired against crvFRAX across FraxBP metapools: $61,534,617.965511166
TVL paired against 3Crv across 3Pool metapools: $495,541,717.5670791
Ratio of paired liquidity of FraxBP to 3Pool: 12.4%


/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [181]:
def chart_metapool_volume(df): 
    return (
        alt.Chart(df)
        .mark_bar()
        .encode(
            x=alt.X("yearmonth(timestamp):T", axis=alt.Axis(labelAlign="center")), 
            y="volume_usd:Q", 
            tooltip=["timestamp", alt.Tooltip("volume_usd", format="$,d")]
        )
    ) 

In [182]:
df_fraxbp_vol_agg = df_fraxbp_volume.groupby(pd.Grouper(key="timestamp", freq="M")).volume_usd.mean().reset_index()
df_3pool_vol_agg = df_3pool_volume.groupby(pd.Grouper(key="timestamp", freq="M")).volume_usd.mean().reset_index()
df_3pool_vol_agg = df_3pool_vol_agg.loc[df_3pool_vol_agg.timestamp >= df_fraxbp_vol_agg.timestamp.min()]

dmax = df_fraxbp_vol_agg.timestamp.max()
vol_frax = df_fraxbp_vol_agg.loc[df_fraxbp_vol_agg.timestamp == dmax].volume_usd.max()
vol_3pool = df_3pool_vol_agg.loc[df_3pool_vol_agg.timestamp == dmax].volume_usd.max()

print(f"Average volume for FraxBP metapools for month ending on {dmax.date()}: ${vol_frax:,}")
print(f"Average volume for 3Pool metapools for month ending on {dmax.date()}: ${vol_3pool:,}")
print(f"Ratio of volume between FraxBP metapools and 3Pool metapools: {100 * (vol_frax / vol_3pool):.1f}%")

(
    chart_metapool_volume(df_fraxbp_vol_agg).properties(title="Volume (Monthly): FraxBP Metapools") | 
    chart_metapool_volume(df_3pool_vol_agg).properties(title="Volume (Monthly): 3Pool Metapools")
)

Average volume for FraxBP metapools for month ending on 2023-01-31: $95,252.01834824336
Average volume for 3Pool metapools for month ending on 2023-01-31: $194,575.59845642984
Ratio of volume between FraxBP metapools and 3Pool metapools: 49.0%


/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)